<a href="https://colab.research.google.com/github/shown5/HandsOnLLM/blob/main/chap10_create_embedding_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datasets import load_dataset

# GLUE からMNLIデータセットを読み込む
# 0 = 合意、１ = 中立、２ = 矛盾
train_dataset = load_dataset(
    "glue", "mnli", split="train"
).select(range(50_000))
train_dataset = traindataset.remove_columns("idx")